In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os
from scripts.GetSummary import summary

Let's use the summary function to collect the data from all participants

In [2]:
participants = []
baseDir = 'data/participantdata'

for folder in os.listdir(baseDir):
    path = os.path.join(baseDir, folder)
    if os.path.isdir(path):
        try:
            summ = summary(path).reset_index()
            summ['participant'] = folder
            summ = summ.set_index('participant')
            participants.append(summ)
        except Exception as e:
            print(f"Skipping {folder}: {e}")

summDF = pd.concat(participants)
summDF.head()

Skipping reldom2_s041: "['Order', 'Type'] not in index"


,Condition,accuracy,RT
participant,,,
reldom2_s001,FO_audwhat,1.00000,1.015675
reldom2_s001,FO_audwhere,0.78125,1.720722
reldom2_s001,FO_viswhat,1.00000,1.296370
reldom2_s001,FO_viswhere,0.96875,1.341634
reldom2_s001,SO_audwhat,1.00000,0.808944


Now, let's pivot our dataframe to get a 16D vector for each participant.

In [3]:
pivot = summDF.pivot_table(
    index='participant',
    columns=['Condition'],
    values=['accuracy', 'RT']
)

pivot.columns = [f'{condition}_{stat}' for stat, condition in pivot.columns]
df = pivot
df.head()

,FO_audwhat_RT,FO_audwhere_RT,FO_viswhat_RT,FO_viswhere_RT,SO_audwhat_RT,SO_audwhere_RT,SO_viswhat_RT,SO_viswhere_RT,FO_audwhat_accuracy,FO_audwhere_accuracy,FO_viswhat_accuracy,FO_viswhere_accuracy,SO_audwhat_accuracy,SO_audwhere_accuracy,SO_viswhat_accuracy,SO_viswhere_accuracy
participant,,,,,,,,,,,,,,,,
reldom2_s001,1.015675,1.720722,1.296370,1.341634,0.808944,1.606454,1.251278,1.128000,1.00000,0.78125,1.00000,0.968750,1.0000,0.7500,0.8750,1.000
reldom2_s002,1.349290,1.356664,1.165648,1.058390,1.461668,1.584935,1.322362,1.095812,0.75000,0.81250,0.93750,0.935484,0.8750,0.6875,0.9375,0.875
reldom2_s003,1.077459,1.149573,0.877843,0.902444,1.103434,1.035821,0.832946,0.766390,0.93750,1.00000,1.00000,1.000000,0.7500,1.0000,1.0000,1.000
reldom2_s004,0.960171,1.142065,1.030101,0.832803,1.052119,1.008416,0.955326,0.707979,0.96875,1.00000,1.00000,0.968750,1.0000,1.0000,1.0000,1.000
reldom2_s005,1.505764,1.259297,1.154793,1.004116,1.255591,0.999182,0.895621,0.822283,0.81250,0.93750,0.96875,0.937500,0.9375,1.0000,0.9375,0.875


Now, let's filter out participants who have average accuracies on any of the tasks less than 0.5.

In [4]:
accCols = [i for i in df.columns if i.endswith('_accuracy')]
df = df[df[accCols].min(axis=1) >= 0.5]
df.head()

,FO_audwhat_RT,FO_audwhere_RT,FO_viswhat_RT,FO_viswhere_RT,SO_audwhat_RT,SO_audwhere_RT,SO_viswhat_RT,SO_viswhere_RT,FO_audwhat_accuracy,FO_audwhere_accuracy,FO_viswhat_accuracy,FO_viswhere_accuracy,SO_audwhat_accuracy,SO_audwhere_accuracy,SO_viswhat_accuracy,SO_viswhere_accuracy
participant,,,,,,,,,,,,,,,,
reldom2_s001,1.015675,1.720722,1.296370,1.341634,0.808944,1.606454,1.251278,1.128000,1.00000,0.78125,1.00000,0.968750,1.0000,0.7500,0.8750,1.000
reldom2_s002,1.349290,1.356664,1.165648,1.058390,1.461668,1.584935,1.322362,1.095812,0.75000,0.81250,0.93750,0.935484,0.8750,0.6875,0.9375,0.875
reldom2_s003,1.077459,1.149573,0.877843,0.902444,1.103434,1.035821,0.832946,0.766390,0.93750,1.00000,1.00000,1.000000,0.7500,1.0000,1.0000,1.000
reldom2_s004,0.960171,1.142065,1.030101,0.832803,1.052119,1.008416,0.955326,0.707979,0.96875,1.00000,1.00000,0.968750,1.0000,1.0000,1.0000,1.000
reldom2_s005,1.505764,1.259297,1.154793,1.004116,1.255591,0.999182,0.895621,0.822283,0.81250,0.93750,0.96875,0.937500,0.9375,1.0000,0.9375,0.875


Assuming k=3, let's implement clustering through KMeans

In [6]:
scaled = StandardScaler().fit_transform(df)

kmeans = KMeans(n_clusters=3, n_init='auto')
labels = kmeans.fit_predict(scaled)

df['cluster'] = labels
df.head()


/var/folders/kc/1tvt60ts6xj710lg4sdb4khr0000gn/T/ipykernel_33157/503809789.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'] = labels


,FO_audwhat_RT,FO_audwhere_RT,FO_viswhat_RT,FO_viswhere_RT,SO_audwhat_RT,SO_audwhere_RT,SO_viswhat_RT,SO_viswhere_RT,FO_audwhat_accuracy,FO_audwhere_accuracy,FO_viswhat_accuracy,FO_viswhere_accuracy,SO_audwhat_accuracy,SO_audwhere_accuracy,SO_viswhat_accuracy,SO_viswhere_accuracy,cluster
participant,,,,,,,,,,,,,,,,,
reldom2_s001,1.015675,1.720722,1.296370,1.341634,0.808944,1.606454,1.251278,1.128000,1.00000,0.78125,1.00000,0.968750,1.0000,0.7500,0.8750,1.000,0
reldom2_s002,1.349290,1.356664,1.165648,1.058390,1.461668,1.584935,1.322362,1.095812,0.75000,0.81250,0.93750,0.935484,0.8750,0.6875,0.9375,0.875,2
reldom2_s003,1.077459,1.149573,0.877843,0.902444,1.103434,1.035821,0.832946,0.766390,0.93750,1.00000,1.00000,1.000000,0.7500,1.0000,1.0000,1.000,1
reldom2_s004,0.960171,1.142065,1.030101,0.832803,1.052119,1.008416,0.955326,0.707979,0.96875,1.00000,1.00000,0.968750,1.0000,1.0000,1.0000,1.000,1
reldom2_s005,1.505764,1.259297,1.154793,1.004116,1.255591,0.999182,0.895621,0.822283,0.81250,0.93750,0.96875,0.937500,0.9375,1.0000,0.9375,0.875,0
